In [59]:
import pandas as pd
from bs4 import BeautifulSoup

pd.options.mode.copy_on_write = True

In [335]:
df = pd.read_csv('csv/house_price_raw.csv')
df.columns = ['PriceTitle', 'Location', 'Size', 'Agent', 'Link', 'Desc']
df = df.drop_duplicates(subset=['PriceTitle', 'Location', 'Size', 'Link'])
df.describe()

,PriceTitle,Location,Size,Agent,Link,Desc
count,21939,21939,21611,18590,21939,21939
unique,2102,168,2527,381,21938,19678
top,Rp 8 Juta /m²,"Dago, Bandung",1000 m²,Pakar Mulia Property logo,/properti/bandung/las4006765/,Tanah Komersial Cocok Untuk Perdagangan Dan Lo...
freq,551,1425,264,1680,2,28


In [342]:
def to_numeric_term(s: str):
    s = s.lower()
    if s == 'miliar': return 1e9
    if s == 'juta': return 1e6
    if s == 'ribu': return 1e3

def assign_price_per_m2(df_inp: pd.DataFrame):
    # separate per meter2 price and total price
    str_m2 = 'Rp (.*?) /m²'
    str_total = 'Rp (.*?) Total'
    df = df_inp['PriceTitle'].str.extract(f'{str_m2}|{str_total}')
    df.columns = ['price_per_m2_raw', 'price_total_raw']
    df = df.assign(total_size=df_inp['Size'], price_per_m2=[0.0 for _ in range(len(df))])

    # convert per meter2 price to numeric
    pm2 = df['price_per_m2_raw']
    pm2_f : pd.Series = pm2[pm2.notnull()]
    pm2_f = pm2_f.str.extract('(.*?) ([a-zA-Z]*)')
    pm2_f.columns = ["pt1", "pt2"]
    pm2_f["pt1"] = pm2_f["pt1"].str.replace(',','.').astype(float)
    pm2_f["pt2"] = pm2_f["pt2"].apply(to_numeric_term)
    pm2_f = pm2_f["pt1"] * pm2_f["pt2"]
    df.loc[pm2.notnull(), 'price_per_m2'] = pm2_f

    # convert total price to numeric then divide by total size
    ptotal = df['price_total_raw']
    size = df_inp[ptotal.notnull()]['Size'].str.extract('(.*?) m²')
    size = size[0].apply(float)
    ptotal_f : pd.Series = ptotal[ptotal.notnull()]
    ptotal_f = ptotal_f.str.extract('(.*?) ([a-zA-Z]*)')
    ptotal_f.columns = ["pt1", "pt2"]
    ptotal_f["pt1"] = ptotal_f["pt1"].str.replace(',','.').astype(float)
    ptotal_f["pt2"] = ptotal_f["pt2"].apply(to_numeric_term)
    ptotal_f = ptotal_f["pt1"] * ptotal_f["pt2"] / size
    df.loc[ptotal.notnull(), 'price_per_m2'] = ptotal_f

    # filter NaN
    df.dropna(subset=['price_per_m2'], inplace=True)

    # filter less than IDR 50k
    df = df[df['price_per_m2'] >= 5e4]

    temp = df_inp[df_inp.index.isin(df.index)]
    df = temp.assign(price_per_m2=df['price_per_m2'])
    return df

df = assign_price_per_m2(df)

In [345]:
df[(df['price_per_m2'] <= 2e5)].describe(include='all')
df[(df['Location'] == 'Arcamanik, Bandung') & (df['price_per_m2'] > 1e6)].describe()
df.describe(include='all')

,price_per_m2
count,5.650000e+02
mean,5.922802e+07
std,4.924626e+08
min,1.300000e+06
25%,4.300000e+06
50%,5.000000e+06
75%,6.500000e+06
max,6.000000e+09
